## 과제 목표
1. Transformer의 multi-head self-attention의 이해 및 구현

# 환경 세팅

본 실습에서 사용할 라이브러리를 import 하겠습니다.

In [20]:
from torch import nn
from torch.nn import functional as F
from tqdm import tqdm

import torch
import math

# 데이터 전처리

본 실습에서 사용할 가상의 데이터를 만들겠습니다. 총 10개 시퀀스의 데이터가 이미 토큰화가 진행되어 주어졌다 가정하겠습니다.

In [21]:
data = [
  [62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75],
  [60, 96, 51, 32, 90],
  [35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54],
  [75, 51],
  [66, 88, 98, 47],
  [21, 39, 10, 64, 21],
  [98],
  [77, 65, 51, 77, 19, 15, 35, 19, 23, 97, 50, 46, 53, 42, 45, 91, 66, 3, 43, 10],
  [70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34],
  [20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43]
]

## 1. Padding 함수 구현하기

어텐션 연산에 들어갈 데이터의 전처리를 진행하겠습니다. 본 실습의 전처리 과정에서는 데이터의 길이를 균일하게 맞출 padding 함수를 구현합니다.

배치 내 데이터 중 최대 길이에 맞도록 다른 데이터의 마지막에 임의의 패딩 값을 패딩해 길이를 맞추는 함수입니다.

In [22]:
############################################################################
# Req 3-1: Padding 함수 구현하기                                             #
############################################################################

def padding(data, pad_value=0):
    """
    Args:
        data (list[list[int]]): data sequence
        pad_value (int): value to pad
    Returns:
        data (list[list[int]]): padded data sequence
        max_len (int): maximum length of intput data
    """
    ################################################################################
    # TODO: 배치 내 데이터 중 최대 길이에 맞추어 다른 데이터의 뒷부분에 패딩 값을 붙여        #
    # 길이를 동일하게 맞추어 반환함                                                     #
    ################################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    # 1) 가장 긴 데이터의 길이 추출
    # 2) 모든 데이터를 돌면서 1에서 구한 길이에 맞게 뒷부분을 pad_value로 채우기

    # 1) 가장 긴 데이터의 길이 추출
    max_len = max(len(seq) for seq in data)
    # 2) 모든 데이터를 돌면서 1에서 구한 길이에 맞게 뒷부분을 pad_value로 채우기
    for i in range(len(data)):
        current_len = len(data[i])
        if current_len < max_len:
            # 패딩을 추가하여 길이를 맞춤
            data[i] += [pad_value] * (max_len - current_len)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ################################################################################
    #                                 END OF YOUR CODE                             #
    ################################################################################

    return data, max_len

아래 코드로 패딩 결과를 확인할 수 있습니다.

In [23]:
pad_value = 0 # 패딩(Padding)은 제로(zero) 패딩으로, 빈 공간을 0으로 채움.
data, max_len = padding(data)
print(f"Maximum sequence length: {max_len}")
for d in data:
    print(d)

Maximum sequence length: 20
[62, 13, 47, 39, 78, 33, 56, 13, 39, 29, 44, 86, 71, 36, 18, 75, 0, 0, 0, 0]
[60, 96, 51, 32, 90, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[35, 45, 48, 65, 91, 99, 92, 10, 3, 21, 54, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[75, 51, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[66, 88, 98, 47, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[21, 39, 10, 64, 21, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[98, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[77, 65, 51, 77, 19, 15, 35, 19, 23, 97, 50, 46, 53, 42, 45, 91, 66, 3, 43, 10]
[70, 64, 98, 25, 99, 53, 4, 13, 69, 62, 66, 76, 15, 75, 45, 34, 0, 0, 0, 0]
[20, 64, 81, 35, 76, 85, 1, 62, 8, 45, 99, 77, 19, 43, 0, 0, 0, 0, 0, 0]


# Single-head Self-Attention 구현하기

Multi-head attention을 구현하기 앞서, 먼저 single-head attention을 구현해보겠습니다. 아래는 single-head self-attention을 위한 설정 값입니다.

In [24]:
# 한 데이터의 최대 길이(vocab_size)는 100이라 가정.
vocab_size = 10000  # 어휘의 크기

# hidden size of model
d_k = 64            # 임베딩 차원

## 2. Embedding 및 Query, Key, Value Vector 구하기

토큰화된 입력 데이터가 임베딩 과정을 거쳐 임베딩 공간의 `d_k` 차원으로 projection되어 임베딩 벡터가 되도록 임베딩 레이어를 정의합니다. PyTorch의 `nn.Embedding` 모듈을 이용할 수 있습니다.

- 참고자료 (PyTorch Emedding): https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html

이후, 임베딩 벡터를 각각 Query, Key, Value 벡터로 변환하는 Linear layer를 정의합니다. `d_k` 차원의 임베딩 벡터를 각각 `d_k` 차원을 가지는 3개의 벡터로 만들어야 합니다. PyTorch의 `nn.Linear` 모듈을 이용할 수 있습니다.

In [25]:
############################################################################
# Req 3-2: Embedding 및 Query, Key, Value Vector 구하기                     #
############################################################################

################################################################################
# TODO: Embedding 레이어를 정의하고, 입력 데이터를 임베딩화함                         #
# 또한 Q, K, V를 위한 레이어를 정의하고, 임베딩 벡터를 각각 Q, K, V 벡터로 변환함       #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
# 1) vocab_size 개수의 데이터 각각을 d_k 차원으로 표현할 임베딩을 할 레이어 정의
# 2) 임베딩 레이어를 사용하여 입력 데이터를 임베딩 벡터로 projection
# 3) d_k 차원의 Q, K, V 벡터를 만들 레이어 정의
# 4) 임베딩 벡터를 Q, K, V 벡터로 변환

# vocab_size 개수의 데이터 각각을 d_k 차원으로 표현할 임베딩을 할 레이어 정의
embedding_layer = nn.Embedding(vocab_size, d_k)

# 임베딩 레이어를 사용하여 입력 데이터를 임베딩 벡터로 projection
# B: Batch, L: Max length of sequence
batch = torch.LongTensor(data) # (B, L)
batch_emb = embedding_layer(batch)  # (B, L, d_k)
print(f"Shape of embedding: {batch_emb.shape}")

# d_k 차원의 Q, K, V 벡터를 만들 레이어 정의
w_q = nn.Linear(d_k, d_k)  # Q를 위한 Linear layer
w_k = nn.Linear(d_k, d_k)  # K를 위한 Linear layer
w_v = nn.Linear(d_k, d_k)  # V를 위한 Linear layer

# 임베딩 벡터를 Q, K, V 벡터로 변환
q = w_q(batch_emb)  # (B, L, d_k)
k = w_k(batch_emb)  # (B, L, d_k)
v = w_v(batch_emb)  # (B, L, d_k)
print(f"Shape of Q, K, V: {q.shape, k.shape, v.shape}")

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

Shape of embedding: torch.Size([10, 20, 64])
Shape of Q, K, V: (torch.Size([10, 20, 64]), torch.Size([10, 20, 64]), torch.Size([10, 20, 64]))


## 3. Scaled Dot-Product Self-Attention 구현하기

![scaled-dot-product-atten](https://velog.velcdn.com/images/glad415/post/1645abbb-e260-4b82-ab1d-c8aa134ea8b7/image.png)

각 head에서 진행하는 Q, K, V의 Self-attention을 구현하겠습니다.

- 가장 먼저, 쿼리와 키(의 전치) 벡터를 곱한 뒤, 벡터 차원의 제곱근으로 나눕니다. 해당 과정은 같은 sequence 내에 서로 다른 token들에게 얼마나 가중치를 두고 attention을 해야하는가를 연산합니다.
- 그 값에 softmax 함수를 취합니다.
- 그 값에 밸류 벡터를 곱하여 최종 attention matrix를 얻습니다.

In [26]:
############################################################################
# Req 3-3: Scaled Dot-Product Self-Attention 구현하기                        #
############################################################################

################################################################################
# TODO: 수식에 맞추어 scaled dot-product self-attention을 구현함                   #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
# 1) Query 벡터와 Key 벡터의 전치를 곱하고, 벡터 차원의 제곱근으로 나눔 (=(Q x K^T) / sqrt(d_k))
# 2) 위 값에 softmax를 취함. row-wise이기 때문에 dim은 -1 로 적용할 것.
# 3) Value 벡터를 곱해 최종 attention value 계산

# Query 벡터와 Key 벡터의 전치를 곱하고, 벡터 차원의 제곱근으로 나눔 (=(Q x K^T) / sqrt(d_k))
# Output shape: (B, L, L)
# 1) Query 벡터와 Key 벡터의 전치를 곱하고, 벡터 차원의 제곱근으로 나눔 (=(Q x K^T) / sqrt(d_k))
attn_scores = torch.matmul(q, k.transpose(-2, -1)) / (d_k ** 0.5)  # (B, L, L)
print(f"Shape of attn_scores: {attn_scores.shape}")


# 위 값에 softmax를 취함. row-wise이기 때문에 dim은 -1 로 적용할 것.
# Output shape: (B, L, L)
attn_dists = F.softmax(attn_scores, dim=-1)  # (B, L, L)
print(f"Shape of attn_dists: {attn_dists.shape}")

# Value 벡터를 곱해 최종 attention value 계산
# Output shape: (B, L, d_k)
attn_values = torch.matmul(attn_dists, v)  # (B, L, d_k)
print(f"Shape of attn_values: {attn_values.shape}")

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

Shape of attn_scores: torch.Size([10, 20, 20])
Shape of attn_dists: torch.Size([10, 20, 20])
Shape of attn_values: torch.Size([10, 20, 64])


# Multi-Head Self-Attention 구현하기

![multi-head-attn](https://velog.velcdn.com/images%2Fcha-suyeon%2Fpost%2F2e70e601-e268-4b55-a8c9-11b3ff145d92%2Fimage.png)

Single-head 코드를 바탕으로 multi-head self-attention을 구현하겠습니다. 전체적으론 아래 내용이 변경 혹은 추가되어야 합니다.

- Q, K, V를 (임베딩 벡터 차원) x (head 개수) 차원으로 projection한 뒤 각 head 개수로 쪼개 사용합니다.
    - 임베딩 과정에서 입력 데이터를 임베딩 공간의 `d_model` 차원으로 projection합니다.
    - 해당 `d_model` 값은 어텐션 헤드의 개수(`n_heads`)로 나누어 떨어져야 합니다.
- 각 헤드의 연산을 한 뒤에는 각 헤드별로 가중치를 곱해 최종 attention value를 구합니다.

아래는 multi-head self-attention 구현에 사용할 설정 값입니다.

In [27]:
# 한 데이터의 최대 길이(vocab_size)는 100이라 가정.
vocab_size = 100

# hidden size of model
d_k = 64

# hidden size of model
d_model = 512

# number of attention heads
num_heads = 8

## 4. Embedding 및 Query, Key, Value Vector 구하기

- Embedding 시 주의사항
  - Single-Head와 마찬가지로 데이터를 임베딩 벡터로 전환해야 합니다. 단, 해당 임베딩 벡터를 나중엔 헤드의 수대로 쪼개야 하기 때문에, 이 부분을 유의합니다.
- Q, K, V 벡터 변환 시 주의사항
  - 이론적으로는 multi-head attention을 수행할 때, input을 각각 다른 head 개수만큼의 `w_q`, `w_k`, `w_v`로 linear transformation하여 각각 여러 번의 attention을 수행한 후 concat하고 linear transformation을 수행합니다.
  - 하지만 구현에서는 $W^Q$\, $W^K$\, $W^V$ 한 개씩만 사용합니다. 여러 헤드의 각 Q, K, V 벡터들을 한 번에 가진 긴 각 Q, K, V 벡터를 만드는 셈입니다.
  - 따라서 각 linear layers `w_q`, `w_k`, `w_v` 는 `d_model` 차원의 입력 임베딩 벡터를 `d_model` 차원의 Q, K, V로 만들어야 합니다.
  - 그리고 이렇게 긴 (여러 헤드의 값을 한 번에 가진) Q, K, V 벡터를 각 헤드만큼씩 쪼개야 합니다. 앞선 single-head attention 때와 마찬가지로, `d_k`는 각 헤드의 벡터의 차원으로, `d_model`이 헤드 개수 만큼씩 쪼개져 들어갑니다. 즉, 각 Q, K, V 벡터를 `d_model` → (num_heads, d_k)` 형태로 변경하는 작업을 합니다.

In [28]:
############################################################################
# Req 3-4: Embedding 및 Query, Key, Value Vector 구하기                      #
############################################################################

################################################################################
# TODO: Embedding 레이어를 정의하고, 입력 데이터를 임베딩화함                         #
# 또한 Q, K, V를 위한 레이어를 정의하고, 임베딩 벡터를 각각 Q, K, V 벡터로 변환함       #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
# 1) vocab_size 개수의 데이터 각각을 d_model 차원으로 표현할 임베딩을 할 레이어 정의
# 2) 임베딩 레이어를 사용하여 입력 데이터를 임베딩 벡터로 projection
# 3) d_model 차원의 Q, K, V 벡터를 만들 레이어 정의
# 4) 임베딩 벡터를 Q, K, V 벡터로 변환
# 5) d_k 차원 계산
# 6) Q, K, V 벡터를 각 헤드만큼 쪼갬 (각 헤드의 각 벡터 차원은 d_k)
# 7) Self-attention 연산을 위하여 각 헤드가 (L, d_k) 행렬을 갖도록 축을 transpose

# vocab_size 개수의 데이터 각각을 d_model 차원으로 표현할 임베딩을 할 레이어 정의
embedding_layer = nn.Embedding(vocab_size, d_model)

# 임베딩 레이어를 사용하여 입력 데이터를 임베딩 벡터로 projection
# B: Batch, L: Max length of sequence
batch = torch.LongTensor(data) # (B, L)
batch_size = batch.shape[0]
batch_emb = embedding_layer(batch)  # (B, L, d_model)
print(f"Shape of embedding: {batch_emb.shape}")

# d_k 차원의 Q, K, V 벡터를 만들 레이어 정의
w_q = nn.Linear(d_model, d_model)  # Q를 위한 Linear layer
w_k = nn.Linear(d_model, d_model)  # K를 위한 Linear layer
w_v = nn.Linear(d_model, d_model)  # V를 위한 Linear layer

# 임베딩 벡터를 Q, K, V 벡터로 변환
q = w_q(batch_emb)  # (B, L, d_model)
k = w_k(batch_emb)  # (B, L, d_model)
v = w_v(batch_emb)  # (B, L, d_model)
print(f"Shape of Q, K, V: {q.shape, k.shape, v.shape}")

# d_k 차원 계산
d_k = d_model // num_heads #"""Write your code"""

# Q, K, V 벡터를 각 헤드만큼 쪼갬 (각 헤드의 각 벡터 차원은 d_k)
q = q.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
k = k.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)
v = v.view(batch_size, -1, num_heads, d_k)  # (B, L, num_heads, d_k)

# Self-attention 연산을 위하여 각 헤드가 (L, d_k) 행렬을 갖도록 축을 transpose
q = q.transpose(1, 2)  # (B, num_heads, L, d_k)
k = k.transpose(1, 2)  # (B, num_heads, L, d_k)
v = v.transpose(1, 2)  # (B, num_heads, L, d_k)
print(f"Shape of Q, K, V: {q.shape, k.shape, v.shape}")

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

Shape of embedding: torch.Size([10, 20, 512])
Shape of Q, K, V: (torch.Size([10, 20, 512]), torch.Size([10, 20, 512]), torch.Size([10, 20, 512]))
Shape of Q, K, V: (torch.Size([10, 8, 20, 64]), torch.Size([10, 8, 20, 64]), torch.Size([10, 8, 20, 64]))


## 5. Scaled Dot-Product Self-Attention 구현하기

해당 부분은 각 head에서 진행되는 attention 연산으로, 코드는 single-head와 동일합니다. 그러나 해당 연산이 이루어지는 matrix의 형태가 다릅니다.

In [29]:
############################################################################
# Req 3-5: Scaled Dot-Product Self-Attention 구현하기                        #
############################################################################

################################################################################
# TODO: 수식에 맞추어 scaled dot-product self-attention을 구현함                   #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
# 1) Query 벡터와 Key 벡터의 전치를 곱하고, 벡터 차원의 제곱근으로 나눔 (=(Q x K^T) / sqrt(d_k))
# 2) 위 값에 softmax를 취함. row-wise이기 때문에 dim은 -1 로 적용할 것.
# 3) Value 벡터를 곱해 최종 attention value 계산

# Query 벡터와 Key 벡터의 전치를 곱하고, 벡터 차원의 제곱근으로 나눔 (=(Q x K^T) / sqrt(d_k))
# Output shape - (B, num_heads, L, L)
attn_scores = torch.matmul(q, k.transpose(-2, -1)) / (d_k ** 0.5)  # (B, L, L)
print(f"Shape of attn_scores: {attn_scores.shape}")

# 위 값에 softmax를 취함. row-wise이기 때문에 dim은 -1 로 적용할 것.
# Output shape: (B, num_heads, L, L)
attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)
print(f"Shape of attn_dists: {attn_dists.shape}")

# Value 벡터를 곱해 최종 attention value 계산
# Output shape: (B, num_heads, L, d_k)
attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)
print(f"Shape of attn_values: {attn_values.shape}")

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

Shape of attn_scores: torch.Size([10, 8, 20, 20])
Shape of attn_dists: torch.Size([10, 8, 20, 20])
Shape of attn_values: torch.Size([10, 8, 20, 64])


## 6. Attention heads의 결과물 병합하기

각 attention head의 결과물을 concatenate해 병합하고, head 별로 정해진 가중치로 linear projection하여 최종 출력을 결정합니다.
이 linear projection은 서로 다른 의미로 focusing된 각 head의 self-attention 정보를 합치는 역할을 합니다.

In [36]:
############################################################################
# Req 3-6: Attention heads의 결과물 병합하기                                  #
############################################################################

################################################################################
# TODO: 각 attention head의 결과물을 병합하고 head 별 가중치로 projection하여        #
# 최종 attention 결과를 구함                                                     #
################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
# 1) Scaled dot-product 의 결과(attn_values)를 (B, L, num_heads, d_k) 형태로 축 변경
# 2) 각 head의 점수를 concatenate함
# 3) 각 head마다의 가중치를 부여할 linear layer 정의
# 4) 위 linear projection layer를 통과하여 최종 출력을 계산


# 임베딩 레이어 정의
embedding_layer = nn.Embedding(vocab_size, d_model)
batch_emb = embedding_layer(batch)  # (B, L, d_model)

# 각 헤드의 결과를 위한 임의의 attn_values 생성 (B, num_heads, L, d_k)
attn_values = torch.randn(batch_size, num_heads, batch_emb.shape[1], d_k)  # 예시 (B, num_heads, L, d_k)

# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
# 1) Scaled dot-product 의 결과(attn_values)를 (B, L, num_heads, d_k) 형태로 축 변경
attn_values = attn_values.transpose(1, 2)  # (B, L, num_heads, d_k)

# 2) 각 head의 점수를 concatenate함
# L과 d_k를 동적으로 가져옴
batch_size, L, num_heads, d_k = attn_values.shape  # 여기서 unpacking
attn_values = attn_values.contiguous().view(batch_size, L, -1)  # (B, L, num_heads * d_k)
print(f"Shape of attention values from all heads: {attn_values.shape}")

# 3) 각 head마다의 가중치를 부여할 linear layer 정의
w_0 = nn.Linear(num_heads * d_k, d_model)  # (num_heads * d_k, d_model)

# 4) 위 linear projection layer를 통과하여 최종 출력을 계산
outputs = w_0(attn_values)  # (B, L, d_model)
print(f"Shape of multi-head self-attention result: {outputs.shape}")

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
################################################################################
#                                 END OF YOUR CODE                             #
################################################################################

Shape of attention values from all heads: torch.Size([10, 20, 1280])
Shape of multi-head self-attention result: torch.Size([10, 20, 512])


# 7. Multi-Head Self-Attention의 모듈 클래스 구현

앞선 <Req. 3-4> ~ <Req. 3-6>의 multi-head self-attention 구현 과정을 하나의 모듈 클래스로 만들어 처리를 손쉽게 구현하겠습니다.

해당 클래스의 forward 연산에서는 임베딩 벡터를 입력으로 받아 multi-head self-attention 결과를 출력하도록 합니다.

In [37]:
############################################################################
# Req 3-7: Multi-Head Self-Attention의 모듈 클래스 구현                       #
############################################################################

class MultiheadAttention(nn.Module):
    def __init__(self, d_model, d_k, num_heads):
        super(MultiheadAttention, self).__init__()

        self.d_model = d_model
        self.d_k = d_k
        self.num_heads = num_heads

        ################################################################################
        # TODO: 조건에 맞게 모델의 레이어를 정의함                                          #
        ################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        # 1) 임베딩 벡터를 Q, K, V 벡터로 변환할 레이어 정의
        # 2) 각 헤드의 결과에 가중치를 부여할 linear layer 정의

        # Q, K, V learnable matrices
        # 1) 임베딩 벡터를 Q, K, V 벡터로 변환할 레이어 정의
        self.w_q = nn.Linear(d_model, d_model)  # Q를 위한 Linear layer
        self.w_k = nn.Linear(d_model, d_model)  # K를 위한 Linear layer
        self.w_v = nn.Linear(d_model, d_model)  # V를 위한 Linear layer

        # Linear projection for concatenated outputs
        self.w_o = nn.Linear(d_model, d_model)  # 최종 출력을 위한 Linear layer

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ################################################################################
        #                                 END OF YOUR CODE                             #
        ################################################################################

    def self_attention(self, q, k, v):
        """scaled-dot product attention

        Args:
            q, k, v (torch.Tensor): Query, Key, Value vectors
        Returns:
            attn_values (torch.Tensor): Result of self-attention
        """
        ################################################################################
        # TODO: Scaled dot-product self-attention 연산을 정의함                          #
        ################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        # 1) Query 벡터와 Key 벡터의 전치를 곱하고, 벡터 차원의 제곱근으로 나눔 (=(Q x K^T) / sqrt(d_k))
        # 2) 위 값에 softmax를 취함. row-wise이기 때문에 dim은 -1 로 적용할 것.
        # 3) Value 벡터를 곱해 최종 attention value 계산

        # 1) Query 벡터와 Key 벡터의 전치를 곱하고, 벡터 차원의 제곱근으로 나눔 (=(Q x K^T) / sqrt(d_k))
        attn_scores = torch.matmul(q, k.transpose(-2, -1)) / (self.d_k ** 0.5)  # (B, num_heads, L, L)

        # 2) 위 값에 softmax를 취함. row-wise이기 때문에 dim은 -1로 적용할 것.
        attn_dists = F.softmax(attn_scores, dim=-1)  # (B, num_heads, L, L)

        # 3) Value 벡터를 곱해 최종 attention value 계산
        attn_values = torch.matmul(attn_dists, v)  # (B, num_heads, L, d_k)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ################################################################################
        #                                 END OF YOUR CODE                             #
        ################################################################################

        return attn_values

    def forward(self, batch_emb):
        batch_size = batch_emb.shape[0]

        ################################################################################
        # TODO: Multi-head self-attention 과정을 정의함                                  #
        ################################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        # 1) 입력 데이터를 Q, K, V 벡터로 변환
        # 2) 위 결과를 Head의 수로 분할함
        # 3) 각 head가 (L, d_k)의 matrix를 담당하도록 만듦
        # 4) Scaled dot-product self-attention 연산을 수행함
        # 5) 각 attention head의 결과물을 concatenate해 병합함
        # 6) head 별로 정해진 가중치로 linear projection하여 최종 출력을 결정함

        # 1) 입력 데이터를 Q, K, V 벡터로 변환
        q = self.w_q(batch_emb)  # (B, L, d_model)
        k = self.w_k(batch_emb)  # (B, L, d_model)
        v = self.w_v(batch_emb)  # (B, L, d_model)

        # 2) 위 결과를 Head의 수로 분할함
        q = q.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)  # (B, num_heads, L, d_k)
        k = k.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)  # (B, num_heads, L, d_k)
        v = v.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)  # (B, num_heads, L, d_k)

        # 3) 각 head가 (L, d_k)의 matrix를 담당하도록 만듦
        # (q, k, v)는 이미 (B, num_heads, L, d_k) 형태입니다.

        # 4) Scaled dot-product self-attention 연산을 수행함
        attn_values = self.self_attention(q, k, v)  # (B, num_heads, L, d_k)

        # 5) 각 attention head의 결과물을 concatenate해 병합함
        attn_values = attn_values.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)  # (B, L, d_model)

        # 6) head 별로 정해진 가중치로 linear projection하여 최종 출력을 결정함
        outputs = self.w_o(attn_values)  # (B, L, d_model)

        return outputs

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ################################################################################
        #                                 END OF YOUR CODE                             #
        ################################################################################

아래 코드로 결과를 테스트할 수 있습니다.

In [38]:
multi_attn = MultiheadAttention(d_model, d_k, num_heads)
outputs = multi_attn(batch_emb)  # (B, L, d_model)

print(outputs)
print(outputs.shape)

tensor([[[ 0.1245,  0.0842, -0.0761,  ...,  0.1766,  0.1458,  0.2143],
         [ 0.0253,  0.1277,  0.0245,  ...,  0.0400, -0.0244, -0.1066],
         [-0.1119,  0.2120, -0.1211,  ...,  0.1040,  0.0908, -0.0629],
         ...,
         [-0.1123,  0.1798, -0.0751,  ...,  0.1715,  0.2028, -0.0963],
         [ 0.0614, -0.0875,  0.1310,  ..., -0.2403, -0.0384, -0.0449],
         [ 0.1872,  0.0861, -0.0043,  ..., -0.1211,  0.0783,  0.1106]],

        [[ 0.0555,  0.4623,  0.1180,  ..., -0.0228,  0.1303, -0.1810],
         [-0.0681,  0.2951,  0.2360,  ...,  0.0986,  0.1621, -0.0319],
         [-0.0405,  0.4314, -0.0377,  ..., -0.0159,  0.0746, -0.1757],
         ...,
         [ 0.0230,  0.3590,  0.0697,  ...,  0.0754,  0.1753, -0.1957],
         [ 0.0245,  0.4841,  0.1009,  ...,  0.1807,  0.2318, -0.1447],
         [ 0.0416,  0.3988,  0.1030,  ...,  0.1449,  0.2803, -0.1220]],

        [[-0.0413, -0.0686, -0.0503,  ...,  0.1010, -0.3001,  0.0131],
         [ 0.1307,  0.2541, -0.0031,  ..., -0